In [12]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 23 10:27:32 2018
Frontend Pololu and RCBenchmark controller
@author: Juan Sebastián Villegas
"""
from tkinter import *
from PIL import Image, ImageTk
import serial
import sys
import glob
"""
Functions
"""
"""Checking for ports"""
def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in list(range(256))]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            #print("checking port "+port)
            s = serial.Serial(port, rtscts=True, dsrdtr=True)
            #print("closing port "+port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [13]:
""" Connecting to Pololu and setting Target"""
def openPololu():
    global targ
    targ=s1.get()
    n=4*int(float(targ))
    if n>=1025*4:
        z=bin(n)
    elif n<1025*4:
        z=bin(1025*4)
    else:
        print('Si eres bueno en algo, no lo hagas gratis')
    f=open(serial_ports()[serial_ports().index(portName.get())],mode='wb')
    binary=[132,0,int(z[8:],2),int(z[2:8],2)]
    f.write(bytearray(binary))
    f.close()

In [16]:
class Controller:
    def __init__(self,ttyStr=portName.get(),device=0x0c):
        # Open the command port
        self.usb = serial.Serial(ttyStr)
        # Command lead-in and device number are sent for each Pololu serial command.
        self.PololuCmd = chr(0xaa) + chr(device)
        # Track target position for each servo. The function isMoving() will
        # use the Target vs Current servo position to determine if movement is
        # occuring.  Upto 24 servos on a Maestro, (0-23). Targets start at 0.
        self.Targets = [0] * 24
        # Servo minimum and maximum targets can be restricted to protect components.
        self.Mins = [0] * 24
        self.Maxs = [0] * 24
    def sendCmd(self, cmd):
        cmdStr = self.PololuCmd + cmd
        if PY2:
            self.usb.write(cmdStr)
        else:
            self.usb.write(bytes(cmdStr,'latin-1'))
        # Run a Maestro Script subroutine in the currently active script. Scripts can
        # have multiple subroutines, which get numbered sequentially from 0 on up. Code your
        # Maestro subroutine to either infinitely loop, or just end (return is not valid).
    def runScriptSub(self):
        cmd = chr(0x27) + chr(0)
        # can pass a param with command 0x28
        # cmd = chr(0x28) + chr(subNumber) + chr(lsb) + chr(msb)
        self.sendCmd(cmd)

In [22]:
#m = maestro.Controller(portName.get())
"""
Front End
"""
window=Tk()
window.title('Automation interface')

l1= Label(window,text='Puerto: ')
l1.grid(row=0,column=0)

l2=Label(window,text='Target: ')
l2.grid(row=1,column=0)

l3=Label(window,text='Sequences: ')
l3.grid(row=2,column=0)

portName=StringVar()
COMs=OptionMenu(window,portName,*serial_ports())
COMs.grid(row=0,column=1)
"""
target=IntVar()
e=Entry(window,textvariable=target)
e.grid(row=1,column=1,columnspan=2)
"""
b1=Button(window,text='Start',width=20,
          command=openPololu,state='active')
b1.grid(row=2,column=0)

b2=Button(window,text='Full scale sequence',width=20,
        command=Controller.runScriptSub,state='active')
b2.grid(row=2,column=1,columnspan=2)

s1=Scale(window,length=400,orient=HORIZONTAL,
         resolution=10,from_=1000,to=1500)
s1.grid(row=1,column=1,columnspan=2)

window.mainloop()
